# Lab - Parallel Processing in Python

Parallel processing is a mode of operation where the task is executed simultaneously in multiple processors in the same computer. It is meant to reduce the overall processing time.

However, there is usually a bit of overhead when communicating between processes which can actually increase the overall time taken for small tasks instead of decreasing it.

In python, the `multiprocessing` module is used to run independent parallel processes by using subprocesses (instead of threads). It allows you to leverage multiple processors on a machine (both Windows and Unix), which means, the processes can be run in completely separate memory locations.

By the end of this lab you will know:

* How to structure the code and understand the syntax to enable parallel processing using `multiprocessing`
* How to implement synchronous and asynchronous parallel processing
* How to parallelize a `Pandas` DataFrame
* Solve 3 different use cases with the `multiprocessing.Pool()` interface

Import `multiprocessing` (you may need to install it)

In [2]:
import multiprocessing as mp

Determine the maximum number of parallel processes can you run. Note: you don't want to use all the available cores since your computer needs to process other things as well.

In [3]:
print("Number of processors: ", mp.cpu_count())

Number of processors:  2


## Synchronous and Asynchronous execution

In parallel processing, there are two types of execution models:

* **_Synchronous_** runs the processes in the same order in which they were started. This is achieved by locking the main program until the respective processes are finished.

* **_Asynchronous_**, on the other hand, doesn’t involve locking. As a result, the order of results can get mixed up but usually gets done quicker.

There are 2 main objects in `multiprocessing` to implement parallel execution of a function: The `Pool` Class and the `Process` Class.

### `Pool` Class

* Synchronous execution
    * `Pool.map()` and `Pool.starmap()`
    * `Pool.apply()` not used in lab
    
* Asynchronous execution
    * `Pool.map_async()` and `Pool.starmap_async()`
    * `Pool.apply_async())` not used in lab

### `Process` Class

Let’s take up a typical problem and implement parallelization using the above techniques. In this lab, we stick to the `Pool` class, because it is most convenient to use and serves most common practical applications.

More info on these classes [here]((https://docs.python.org/3/library/multiprocessing.html))

Read about the differences between apply, map, and starmap [here](https://stackoverflow.com/questions/8533318/multiprocessing-pool-when-to-use-apply-apply-async-or-map)

## Example

Given a 2D matrix (or list of lists), count how many numbers are present between a given range in each row. We will transform a matrix into a list of rows. 

Import the following libraries:

In [4]:
import numpy as np
from time import time

Use `np.randon.RandomState(100)` to set a random seed.

In [5]:
np.random.RandomState(100)

RandomState(MT19937) at 0x7F24F81A7D10

Let's create a large numpy array matrix of integers between 0 and 9 with 1,000,000 rows and 5 columns. You can use `np.randon.randint` for this.

In [34]:
large_mat = np.random.randint(10,size=(1000000,5))

Convert the array you created in the previous step to a list using `tolist()`.

In [36]:
large_list = large_mat.tolist()

Explore your array

In [37]:
large_mat[:10,:10]

array([[1, 1, 0, 0, 2],
       [3, 0, 9, 7, 7],
       [6, 0, 7, 3, 5],
       [4, 8, 4, 8, 0],
       [8, 9, 3, 7, 4],
       [8, 3, 1, 5, 3],
       [4, 4, 4, 8, 3],
       [4, 6, 6, 5, 7],
       [3, 2, 5, 4, 3],
       [7, 4, 1, 3, 9]])

Check that the length of your list is 1,000,000

In [38]:
len(large_list)

1000000

### Implement solution without parallelization

Let’s see how long it takes to compute it without parallelization. For this, we create a function called `howmany_within_range()` and we iterate the function over every row of the matrix. Since we converted the matrix to a list, then we iterate over every element in the list. The function receives all the values on the row (list element) as input and return the count.

In [40]:
def howmany_within_range(row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

Iterate the function `howmany_within_range` over every row in the matrix and measure the time.

In [51]:
import time

start = time.time()
## Fill in code here
results = []
for row in large_list:
    results.append(howmany_within_range(row, 0, 9))
    
end = time.time()
print(end - start)

1.0777387619018555


Print the first 10 results

In [53]:
print(results[:10])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


Check that the length of results is 1,000,000 and your implementation didn't cheat!

In [52]:
len(results)

1000000

## How to parallelize any function?

### CAVEATS

* Parallelizing a function written in the same notebook does not work on Windows. You would have to write the function in a separate script and then import it. We do not need to do that since we are in SageMaker.
* For reasons unknown to us, `Pool.apply()` does not work on Windows or Mac M1 machines. It does work on Linux and Intel Macs. This is also fine since we are working in Linux!

The general way to parallelize any operation is to take a particular function that should be run multiple times and make it run in parallel using different processors.

To do this, you initialize a _Pool_ with n number of processors (or cores) and pass the function you want to parallelize to one of _Pools_ parallization methods.

`multiprocessing.Pool()` provides the `apply()`, `map()` and `starmap()` methods to make any function run in parallel.

Nice! So what’s the difference between `apply()` and `map()`?

Both `apply` and `map` take the function to be "parallelized" as the main argument. But the difference is that `apply()` takes an _args_ argument that accepts the parameters passed to the _function-to-be-parallelized_ as an argument, whereas `map` can take only one _iterable_ as an argument. This is because `apply()` only runs one worker in the pool so this is not parallelization of the function, it is parallelization of a potentially larger system.

So `map()` is really more suitable for simpler iterable operations and also does the job faster because it is using all of the workers in the pool.

We will get to `starmap()` once we see how to parallelize `howmany_within_range()` function with `apply()` and `map()`.

np.array(1000000).tolist()### Parallelizing using `Pool.map()`

`Pool.map()` accepts only _one iterable as argument_. So as a workaround, we modify the `howmany_within_range` function by setting a default to the minimum and maximum parameters to create a new `howmany_within_range_rowonly()` function so it accetps only an _iterable list_ of rows as input. This is not a nice use case of map(), but it clearly shows how it differs from apply().

In [67]:
# Parallelizing using Pool.map()

# Redefine function with only 1 mandatory argument.
def howmany_within_range_rowonly(row):
    return howmany_within_range(row, 0, 10)
results = []
start = time.time()
pool = mp.Pool(mp.cpu_count())

t1 = time.time()
## use the pool.map function
results = pool.map(howmany_within_range_rowonly , large_list)
t2 = time.time()
pool.close()
end = time.time()

print('total time',end - start)
print('time to set up pool',t1 - start)
print('time to multiprocess',t2 - t1)


print(results[:10])

total time 1.811729907989502
time to set up pool 0.0309450626373291
time to multiprocess 1.7806625366210938
[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


### Parallelizing using `Pool.starmap()`

In previous example, we have to redefine `howmany_within_range` function to make couple of parameters to take default values. Using `starmap()`, you can avoid doing this. How you ask?

Like `Pool.map()`, `Pool.starmap()` also accepts only one iterable as argument, but in `starmap()`, each element in that iterable is also a iterable. You can to provide the arguments to the _function-to-be-parallelized_ in the same order in this inner iterable element, will in turn be unpacked during execution.

So effectively, `Pool.starmap()` is like a version of Pool.map() that accepts arguments.

In [68]:
# Parallelizing with Pool.starmap()
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())

# use pool.starmap
pool.starmap(howmany_within_range, [(row, 0, 10) for row in large_list])

pool.close()

print(results[:10])

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


## Asynchronous Parallel Processing

The asynchronous equivalents `apply_async()`, `map_async()` and `starmap_async()` lets you do execute the processes in parallel asynchronously, that is the next process can start as soon as previous one gets over without regard for the starting order. As a result, there is no guarantee that the result will be in the same order as the input.

## Parallelizing with `Pool.starmap_async()`

You saw how `apply_async()` works. Can you imagine and write up an equivalent version for starmap_async and map_async? The implementation is below anyways.

In [70]:
# Parallelizing with Pool.starmap_async()

def howmany_within_range2(i, row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return (i, count)

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

results = pool.starmap_async(howmany_within_range2, [(i, row, 4, 8) for i, row in enumerate(large_list)]).get()

# With map, use `howmany_within_range_rowonly` instead
# results = pool.map_async(howmany_within_range_rowonly, [row for row in data]).get()

pool.close()
print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

[(0, 0), (1, 2), (2, 3), (3, 4), (4, 3), (5, 2), (6, 4), (7, 5), (8, 2), (9, 2)]


# Exercises

## Problem 1

Use `Pool.starmap()` to get the row wise common items in list_a and list_b. Each iteration will be row_i in list_a and list_b:

```
list_a = [[1, 2, 3], [5, 6, 7, 8], [10, 11, 12], [20, 21]]
list_b = [[2, 3, 4, 5], [6, 9, 10], [11, 12, 13, 14], [21, 24, 25]]
```

In [106]:
list_a = [[1, 2, 3], [5, 6, 7, 8], [10, 11, 12], [20, 21]]
list_b = [[2, 3, 4, 5], [6, 9, 10], [11, 12, 13, 14], [21, 24, 25]]
temp_set = set(list_a[0])


In [107]:
list_a = [[1, 2, 3], [5, 6, 7, 8], [10, 11, 12], [20, 21]]
list_b = [[2, 3, 4, 5], [6, 9, 10], [11, 12, 13, 14], [21, 24, 25]]

def common_items(list1, list2):
    return list(set(list1).intersection(set(list2)))

pool = mp.Pool(mp.cpu_count())

prob1_answer = pool.starmap(common_items, [(list_a[i], list_b[i]) for i in range(len(list_a))])

pool.close()

print(prob1_answer)
#poo.starmap()

[[2, 3], [6], [11, 12], [21]]


## Problem 2 

You have three scripts named `script1.py`, `script2.py`, `script3.py` in your folder. Use `Pool.map()` to run each of them in parallel.

You can use the function `subprocess.getstatusoutput()` that will give you the `exit code` and `stdout` from each Linux command. You know how to run a python script in Linux. Now put thoese pieces together to "map" a python script execution.

Each script pauses and prints its name and time paused.

```
$ python script1.py 
process 1 done, slept for 2
```

We will see if the code parallelizes effectively by seeing the time taken to be less than the sum of each individual script. Do the calculation to prove it for yourself.

Save your result to `prob2_answer`, which should contain the `exit code` and `stdout` from each process in a list, which is just the output from `pool.map`.

In [102]:
import subprocess
pool = mp.Pool(mp.cpu_count())
exec_cmds = [f'python script{i}.py' for i in range(1,4)]
subprocess.getstatusoutput('python script1.py')

prob2_answer = pool.starmap(subprocess.getstatusoutput, exec_cmds)

pool.close()

CompletedProcess(args=['python', 'fall-2022-lab3-kianakaslana648\\script1.py'], returncode=2)

b'process 1 done, slept for 4\n'


(0, 'process 1 done, slept for 4')

## Problem 3

**Normalize** each row of the following 2d array (list) to vary between 0 and 1. Each row should be processed separately. That means one min/max for each row. Execute in parallel.

Save your result to `prob3_answer`

Read about normalization options [here](https://stats.stackexchange.com/questions/70801/how-to-normalize-data-to-0-1-range). Use the following formula:

$$z_i=\frac{x_i−min(x)}{max(x)−min(x)}$$

`list_a = [[2, 3, 4, 5], [6, 9, 10, 12], [11, 12, 13, 14], [21, 24, 25, 26], [23402, 234727, 4532, 7534]]`

In [108]:
list_aa = [[2, 3, 4, 5], [6, 9, 10, 12], [11, 12, 13, 14], [21, 24, 25, 26], [23402, 234727, 4532, 7534]]

def norm_z(mylist):
    temp_arr = np.array(mylist)
    return ((temp_arr - temp_arr.min())/(temp_arr.max()-temp_arr.min())).tolist()

pool = mp.Pool(mp.cpu_count())

prob3_answer = pool.map(norm_z, [row for row in list_aa])

pool.close()

print(prob3_answer)

[[0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.5, 0.6666666666666666, 1.0], [0.0, 0.3333333333333333, 0.6666666666666666, 1.0], [0.0, 0.6, 0.8, 1.0], [0.08197397858337496, 1.0, 0.0, 0.013041117313581964]]


## **Save your analytics results to a json object - then add, commit, and push your notebook and json to GitHub!**

In [ ]:
import json
json.dump({'prob1' : str(prob1_answer),
           'prob2' : str(prob2_answer),
           'prob3' : str(prob3_answer)
          }, 
          fp = open('soln.json','w'))